In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import b3d
import sys

sys.path.append(str(b3d.get_root_path()))

In [3]:
from tests.sama4d.video_to_tracks.video_to_tracks_task import VideoToTracksTask
import rerun

In [4]:
path = b3d.get_assets_path() / 'shared_data_bucket/dynamic_SfM/feature_track_data/pan_around_blocks.npz'
ftd_og = b3d.io.FeatureTrackData.load(str(path)).slice_time(20, 40)

In [5]:
rerun.init("multiple_patch_with_reinstantiation")
rerun.connect("127.0.0.1:8812")

In [6]:
task = VideoToTracksTask.from_feature_track_data(ftd_og, has_no_moving_objects=True)
task.visualize_task()

In [32]:
from tests.sama4d.video_to_tracks.solvers.conv_with_reinstantiation import KeypointTracker2DWithReinitialization

solver = KeypointTracker2DWithReinitialization(
    patch_size=9,
    num_tracks=40,
    frames_before_adding_to_active_set=3,
    reinitialize_patches=True,
    culling_error_threshold=60,
    culling_error_ratio_threshold=0.6,
    mindist_for_second_error=4,
    maxdist_for_second_error=40
)

In [78]:
spec = task.get_task_specification()
solution = solver.solve(spec)

In [70]:
import jax
import jax.numpy as jnp
from functools import partial

# @partial(jax.jit, static_argnums=(1,))
def replace_using_elements_from_other_vector(
    x,
    condition_on_x,
    new_values
):
    """
    This is used when we want to replace some values in x
    using elements from new_values.
    This function returns a version of x, where all elements
    not satisfying condition_on_x are replaced with elements
    from new_values.
    The elements of new_values are pulled out in order.
    This is JAX compatible.
    """

    # state = (current_array, idx_in_x, idx_in_new_indices_full)
    state = (x, 0, 0)

    def step(state, _):
        x, xidx, nidx = state
        x = x.at[xidx].set(jnp.where(
            condition_on_x(x[xidx]),
            x[xidx],
            new_values[nidx]
        ))
        new_nidx = jnp.where(
            condition_on_x(x[xidx]), nidx, nidx + 1
        )
        new_xidx = xidx + 1
        return (x, new_xidx, new_nidx), None

    final_state, _ = jax.lax.scan(step, state, jnp.arange(x.size))
    return final_state[0]

In [74]:
replace_using_elements_from_other_vector(
    jnp.arange(10),
    lambda x: x > 5,
    jnp.ones(10, dtype=int)
)

Array([1, 1, 1, 1, 1, 1, 6, 7, 8, 9], dtype=int32)

In [47]:
import jax.numpy as jnp
import jax
jnp.zeros(10).at[jnp.arange(5)].set(
    jax.lax.dynamic_update_slice
    jnp.ones(10))

ValueError: Incompatible shapes for broadcasting: (10,) and requested shape (5,)